# Fields for tournament games

# Informational fields:
* date
* season
* team
* opponent
* win/loss

# Predictor fields
* regular_season_win_pct - just conference and non-conference games
* opponent_regular_season_win_pct
* season_win_pct - includes conference tourney games
* opponent_season_win_pct
* season_ppg - includes conference tourney games
* opponent_season_ppg
* season_ppg_allowed - includes conference tourney games
* opponent_season_ppg_allowed
* last_10_games_win_pct - includes conference tourney games
* opponent_last_10_games_win_pct
* last_5_games_win_pct - includes conference tourney games
* opponent_last_5_games_win_pct
* vs_tourney_team_win_pct - includes conference tourney games
* opponent_vs_tourney_team_win_pct
* games_played_vs_tourney_teams - includes conference tourney games
* opponent_games_played_vs_tourney_teams
* season_adj_offense - includes conference tourney games
* opponent_season_adj_offense
* vs_tourney_team_adj_offense - includes conference tourney games
* opponent_vs_tourney_team_adj_offense
* season_adj_defense - includes conference tourney games
* opponent_season_adj_defense
* vs_tourney_team_adj_defense - includes conference tourney games
* opponent_vs_tourney_team_adj_defense
* season_offensive_efficiency - includes conference tourney games
* opponent_season_offensive_efficiency
* vs_tourney_team_offensive_efficiency - includes conference tourney games
* opponent_vs_tourney_team_offensive_efficiency
* season_effective_field_goal_pct - includes conference tourney games
* opponent_season_effective_field_goal_pct
* vs_tourney_team_effective_field_goal_pct - includes conference tourney games
* opponent_vs_tourney_team_effective_field_goal_pct
* season_turnover_pct - includes conference tourney games
* opponent_season_turnover_pct
* vs_tourney_team_turnover_pct - includes conference tourney games
* opponent_vs_tourney_team_turnover_pct
* season_off_reb_pct - includes conference tourney games
* opponent_season_off_reb_pct
* vs_tourney_team_off_reb_pct - includes conference tourney games
* opponent_vs_tourney_team_off_reb_pct
* season_ft_rate - includes conference tourney games
* opponent_season_ft_rate
* vs_tourney_team_ft_rate - includes conference tourney games
* opponent_vs_tourney_team_ft_rate
* season_allowed_offensive_efficiency - includes conference tourney games
* opponent_season_allowed_offensive_efficiency
* vs_tourney_team_allowed_offensive_efficiency - includes conference tourney games
* opponent_vs_tourney_team_allowed_offensive_efficiency
* season_allowed_effective_field_goal_pct - includes conference tourney games
* opponent_season_allowed_effective_field_goal_pct
* vs_tourney_team_allowed_effective_field_goal_pct - includes conference tourney games
* opponent_vs_tourney_team_allowed_effective_field_goal_pct
* season_allowed_turnover_pct - includes conference tourney games
* opponent_season_allowed_turnover_pct
* vs_tourney_team_allowed_turnover_pct - includes conference tourney games
* opponent_vs_tourney_team_allowed_turnover_pct
* season_allowed_off_reb_pct - includes conference tourney games
* opponent_season_allowed_off_reb_pct
* vs_tourney_team_allowed_off_reb_pct - includes conference tourney games
* opponent_vs_tourney_team_allowed_off_reb_pct
* season_allowed_ft_rate - includes conference tourney games
* opponent_season_allowed_ft_rate
* vs_tourney_team_allowed_ft_rate - includes conference tourney games
* opponent_vs_tourney_team_allowed_ft_rate
* season_allowed_adj_offense - includes conference tourney games
* opponent_season_allowed_adj_offense
* vs_tourney_team_allowed_adj_offense - includes conference tourney games
* opponent_vs_tourney_team_allowed_adj_offense
* season_allowed_adj_defense - includes conference tourney games
* opponent_season_allowed_adj_defense
* vs_tourney_team_allowed_adj_defense - includes conference tourney games
* opponent_vs_tourney_team_allowed_adj_defense

In [2]:
import os
import pandas as pd
import sqlalchemy
import sklearn
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

connection_string = os.getenv("LINODE_CONN_STRING")
connection_string = os.getenv("LINODE_CONN_STRING")
conn = sqlalchemy.create_engine(os.getenv("LINODE_CONN_STRING")).connect()

In [3]:
all_games = pd.read_sql("select * from sports_data.game_stats_v3_2025", conn)

In [4]:
tourney_games = all_games[all_games["game_type"] == "Post-Season Tourney"]
tourney_games = tourney_games[tourney_games['location'] == 'N']

In [5]:
tourney_games.columns

Index(['date', 'game_type', 'team', 'conference', 'opponent', 'location',
       'result', 'adj_offense', 'adj_defense', 'offensive_efficiency',
       'effective_fg_percent', 'turnover_percent', 'offensive_rebound_percent',
       'free_throw_rate', 'opponent_offensive_efficiency',
       'opponent_effective_fg_percent', 'opponent_turnover_percent',
       'opponent_offensive_rebound_percent', 'opponent_free_throw_rate',
       'opponent_conference', 'season', 'Win_Loss', 'score', 'opponent_score'],
      dtype='object')

In [6]:
tourney_games = tourney_games[['date', 'season', 'team', 'opponent', 'Win_Loss']]

In [7]:
all_games['game_type'].unique()

array(['Non-Conference', 'Conference', 'Conference Tourney',
       'Post-Season Tourney'], dtype=object)

In [8]:
regular_season_games = all_games[all_games["game_type"].isin(['Non-Conference', 'Conference'])]
pre_tourney_games = all_games[all_games["game_type"].isin(['Non-Conference', 'Conference', 'Conference Tourney'])]
last_5_non_tourney = all_games[all_games['game_type'] != 'Post-Season Tourney'].sort_values('date').groupby(['team', 'season']).tail(5).sort_values(['team', 'season', 'date'])
last_10_non_tourney = all_games[all_games['game_type'] != 'Post-Season Tourney'].sort_values('date').groupby(['team', 'season']).tail(10).sort_values(['team', 'season', 'date'])
all_games['opp_season'] = all_games.apply(lambda x: f"{x['opponent']}_{x['season']}", axis=1)
tourney_teams = tourney_games.apply(lambda x: f"{x['team']}_{x['season']}", axis=1).unique()
games_vs_tourney_teams = all_games[all_games['opp_season'].isin(tourney_teams)]

In [9]:
working = regular_season_games.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['regular_season_win_pct'] = working['W'] / (working['W'] + working['L'])
regular_season_win_pct = working[['team', 'season', 'regular_season_win_pct']]

In [10]:
tourney_games = tourney_games.merge(regular_season_win_pct, on=['team', 'season'], how='left')

In [11]:
regular_season_win_pct.rename(columns={'regular_season_win_pct': 'opponent_regular_season_win_pct', 'team':'opponent'}, inplace=True)

C:\Users\amarl\AppData\Local\Temp\ipykernel_29388\1859462520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regular_season_win_pct.rename(columns={'regular_season_win_pct': 'opponent_regular_season_win_pct', 'team':'opponent'}, inplace=True)


In [12]:
tourney_games

,date,season,team,opponent,Win_Loss,regular_season_win_pct
0,3/18/08,2008,Coppin St.,Mount St. Mary's,L,0.333333
1,3/18/08,2008,Mount St. Mary's,Coppin St.,W,0.517241
2,3/19/08,2008,Utah,UTEP,W,0.535714
3,3/19/08,2008,UTEP,Utah,L,0.586207
4,3/19/08,2008,UAB,VCU,W,0.709677
...,...,...,...,...,...,...
2381,3/19/15,2015,Wofford,Arkansas,L,0.793103
2382,3/20/15,2015,Northern Iowa,Wyoming,W,0.900000
2383,3/20/15,2015,Wyoming,Northern Iowa,L,0.689655
2384,3/26/15,2015,Arizona,Xavier,W,0.903226


In [13]:
tourney_games = tourney_games.merge(regular_season_win_pct, on=['opponent', 'season'], how='left')

In [14]:
working = pre_tourney_games.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['season_win_pct'] = working['W'] / (working['W'] + working['L'])
season_win_pct = working[['team', 'season', 'season_win_pct']]

In [15]:
tourney_games = tourney_games.merge(season_win_pct, on=['team', 'season'], how='left')

In [16]:
season_win_pct.rename(columns={'season_win_pct': 'opponent_season_win_pct', 'team':'opponent'}, inplace=True)

C:\Users\amarl\AppData\Local\Temp\ipykernel_29388\1093291026.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_win_pct.rename(columns={'season_win_pct': 'opponent_season_win_pct', 'team':'opponent'}, inplace=True)


In [17]:
tourney_games = tourney_games.merge(season_win_pct, on=['opponent', 'season'], how='left')

In [18]:
pre_tourney_games['score'] = pre_tourney_games['score'].astype(int)
season_ppg = pre_tourney_games[['team', 'season','score']].groupby(['team', 'season']).mean().reset_index().rename(columns={'score': 'season_ppg'})

C:\Users\amarl\AppData\Local\Temp\ipykernel_29388\2704502709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_tourney_games['score'] = pre_tourney_games['score'].astype(int)


In [19]:
tourney_games = tourney_games.merge(season_ppg, on=['team', 'season'], how='left')

In [20]:
season_ppg.rename(columns={'season_ppg': 'opponent_season_ppg', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_ppg, on=['opponent', 'season'], how='left')

In [21]:
pre_tourney_games['opponent_score'] = pre_tourney_games['opponent_score'].astype(int)
season_ppg_allowed = pre_tourney_games[['team', 'season','opponent_score']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_score': 'season_ppg_allowed'})

C:\Users\amarl\AppData\Local\Temp\ipykernel_29388\2874491460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pre_tourney_games['opponent_score'] = pre_tourney_games['opponent_score'].astype(int)


In [22]:
season_ppg_std = pre_tourney_games[['team', 'season','score']].groupby(['team', 'season']).std().reset_index().rename(columns={'score': 'season_ppg_std'})
tourney_games = tourney_games.merge(season_ppg_std, on=['team', 'season'], how='left')
season_ppg_std.rename(columns={'season_ppg_std': 'opponent_season_ppg_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_ppg_std, on=['opponent', 'season'], how='left')

In [23]:
season_ppg_allowed_std = pre_tourney_games[['team', 'season','opponent_score']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_score': 'season_ppg_allowed_std'})
tourney_games = tourney_games.merge(season_ppg_allowed_std, on=['team', 'season'], how='left')
season_ppg_allowed_std.rename(columns={'season_ppg_allowed_std': 'opponent_season_ppg_allowed_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_ppg_allowed_std, on=['opponent', 'season'], how='left')

In [24]:
tourney_games = tourney_games.merge(season_ppg_allowed, on=['team', 'season'], how='left')

In [25]:
season_ppg_allowed.rename(columns={'season_ppg_allowed': 'opponent_season_ppg_allowed', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_ppg_allowed, on=['opponent', 'season'], how='left')

In [26]:
working = last_10_non_tourney.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['last_10_win_pct'] = working['W'] / (working['W'] + working['L'])
last_10_win_pct = working[['team', 'season', 'last_10_win_pct']]

In [27]:
tourney_games = tourney_games.merge(last_10_win_pct, on=['team', 'season'], how='left')

In [28]:
last_10_non_tourney['score'] = last_10_non_tourney['score'].astype(int)
last_10_ppg = last_10_non_tourney[['team', 'season','score']].groupby(['team', 'season']).mean().reset_index().rename(columns={'score': 'last_10_ppg'})
tourney_games = tourney_games.merge(last_10_ppg, on=['team', 'season'], how='left')
last_10_ppg.rename(columns={'last_10_ppg': 'opponent_last_10_ppg', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_ppg, on=['opponent', 'season'], how='left')

In [29]:
last_10_ppg_std = last_10_non_tourney[['team', 'season','score']].groupby(['team', 'season']).std().reset_index().rename(columns={'score': 'last_10_ppg_std'})
tourney_games = tourney_games.merge(last_10_ppg_std, on=['team', 'season'], how='left')
last_10_ppg_std.rename(columns={'last_10_ppg_std': 'opponent_last_10_ppg_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_ppg_std, on=['opponent', 'season'], how='left')

In [30]:
last_10_non_tourney['opponent_score'] = last_10_non_tourney['opponent_score'].astype(int)
last_10_ppg_allowed = last_10_non_tourney[['team', 'season','opponent_score']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_score': 'last_10_ppg_allowed'})
tourney_games = tourney_games.merge(last_10_ppg_allowed, on=['team', 'season'], how='left')
last_10_ppg_allowed.rename(columns={'last_10_ppg_allowed': 'opponent_last_10_ppg_allowed', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_ppg_allowed, on=['opponent', 'season'], how='left')

In [31]:
last_10_ppg_allowed_std = last_10_non_tourney[['team', 'season','opponent_score']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_score': 'last_10_ppg_allowed_std'})
tourney_games = tourney_games.merge(last_10_ppg_allowed_std, on=['team', 'season'], how='left')
last_10_ppg_allowed_std.rename(columns={'last_10_ppg_allowed_std': 'opponent_last_10_ppg_allowed_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_ppg_allowed_std, on=['opponent', 'season'], how='left')

In [32]:
last_10_win_pct.rename(columns={'last_10_win_pct': 'opponent_last_10_win_pct', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_win_pct, on=['opponent', 'season'], how='left')

C:\Users\amarl\AppData\Local\Temp\ipykernel_29388\1411312810.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_10_win_pct.rename(columns={'last_10_win_pct': 'opponent_last_10_win_pct', 'team':'opponent'}, inplace=True)


In [33]:
working = last_5_non_tourney.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['last_5_win_pct'] = working['W'] / (working['W'] + working['L'])
last_5_win_pct = working[['team', 'season', 'last_5_win_pct']]

In [34]:
tourney_games = tourney_games.merge(last_5_win_pct, on=['team', 'season'], how='left')

In [35]:
last_5_win_pct.rename(columns={'last_5_win_pct': 'opponent_last_5_win_pct', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_5_win_pct, on=['opponent', 'season'], how='left')

C:\Users\amarl\AppData\Local\Temp\ipykernel_29388\489938241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_5_win_pct.rename(columns={'last_5_win_pct': 'opponent_last_5_win_pct', 'team':'opponent'}, inplace=True)


In [36]:
working = games_vs_tourney_teams.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['vs_tourney_team_win_pct'] = working['W'] / (working['W'] + working['L'])
vs_tourney_team_win_pct = working[['team', 'season', 'vs_tourney_team_win_pct']]

In [37]:
tourney_games = tourney_games.merge(vs_tourney_team_win_pct, on=['team', 'season'], how='left')

In [38]:
vs_tourney_team_win_pct.rename(columns={'vs_tourney_team_win_pct': 'opponent_vs_tourney_team_win_pct', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_win_pct, on=['opponent', 'season'], how='left')

C:\Users\amarl\AppData\Local\Temp\ipykernel_29388\2986988654.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vs_tourney_team_win_pct.rename(columns={'vs_tourney_team_win_pct': 'opponent_vs_tourney_team_win_pct', 'team':'opponent'}, inplace=True)


In [39]:
working = games_vs_tourney_teams.groupby(['team', 'season','Win_Loss'], as_index=False).size()
working = working.pivot_table(index=['team', 'season'], columns='Win_Loss', values='size', fill_value=0).reset_index()[['team', 'season', 'L', 'W']]
working['games_played_vs_tourney_teams'] = (working['W'] + working['L'])
games_played_vs_tourney_teams = working[['team', 'season', 'games_played_vs_tourney_teams']]

In [40]:
tourney_games = tourney_games.merge(games_played_vs_tourney_teams, on=['team', 'season'], how='left')

In [41]:
games_played_vs_tourney_teams.rename(columns={'games_played_vs_tourney_teams': 'opponent_games_played_vs_tourney_teams', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(games_played_vs_tourney_teams, on=['opponent', 'season'], how='left')

C:\Users\amarl\AppData\Local\Temp\ipykernel_29388\3857983212.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  games_played_vs_tourney_teams.rename(columns={'games_played_vs_tourney_teams': 'opponent_games_played_vs_tourney_teams', 'team':'opponent'}, inplace=True)


In [42]:
pre_tourney_games.columns

Index(['date', 'game_type', 'team', 'conference', 'opponent', 'location',
       'result', 'adj_offense', 'adj_defense', 'offensive_efficiency',
       'effective_fg_percent', 'turnover_percent', 'offensive_rebound_percent',
       'free_throw_rate', 'opponent_offensive_efficiency',
       'opponent_effective_fg_percent', 'opponent_turnover_percent',
       'opponent_offensive_rebound_percent', 'opponent_free_throw_rate',
       'opponent_conference', 'season', 'Win_Loss', 'score', 'opponent_score'],
      dtype='object')

In [43]:
season_adj_offense = pre_tourney_games[['team', 'season','adj_offense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_offense': 'season_adj_offense'})

In [44]:
tourney_games = tourney_games.merge(season_adj_offense, on=['team', 'season'], how='left')

In [45]:
season_adj_offense.rename(columns={'season_adj_offense': 'opponent_season_adj_offense', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_adj_offense, on=['opponent', 'season'], how='left')

In [46]:
season_adj_offense_std = pre_tourney_games[['team', 'season','adj_offense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_offense': 'season_adj_offense_std'})
tourney_games = tourney_games.merge(season_adj_offense_std, on=['team', 'season'], how='left')
season_adj_offense_std.rename(columns={'season_adj_offense_std': 'opponent_season_adj_offense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_adj_offense_std, on=['opponent', 'season'], how='left')

In [47]:
last_10_adj_offense = last_10_non_tourney[['team', 'season','adj_offense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_offense': 'last_10_adj_offense'})
tourney_games = tourney_games.merge(last_10_adj_offense, on=['team', 'season'], how='left')
last_10_adj_offense.rename(columns={'last_10_adj_offense': 'opponent_last_10_adj_offense', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_adj_offense, on=['opponent', 'season'], how='left')

In [48]:
last_10_adj_offense_std = last_10_non_tourney[['team', 'season','adj_offense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_offense': 'last_10_adj_offense_std'})
tourney_games = tourney_games.merge(last_10_adj_offense_std, on=['team', 'season'], how='left')
last_10_adj_offense_std.rename(columns={'last_10_adj_offense_std': 'opponent_last_10_adj_offense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_adj_offense_std, on=['opponent', 'season'], how='left')

In [49]:
season_adj_defense = pre_tourney_games[['team', 'season','adj_defense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_defense': 'season_adj_defense'})

In [50]:
tourney_games = tourney_games.merge(season_adj_defense, on=['team', 'season'], how='left')

In [51]:
season_adj_defense.rename(columns={'season_adj_defense': 'opponent_season_adj_defense', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_adj_defense, on=['opponent', 'season'], how='left')

In [52]:
season_adj_defense_std = pre_tourney_games[['team', 'season','adj_defense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_defense': 'season_adj_defense_std'})
tourney_games = tourney_games.merge(season_adj_defense_std, on=['team', 'season'], how='left')
season_adj_defense_std.rename(columns={'season_adj_defense_std': 'opponent_season_adj_defense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_adj_defense_std, on=['opponent', 'season'], how='left')

In [53]:
vs_tourney_team_adj_offense = games_vs_tourney_teams[['team', 'season','adj_offense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_offense': 'vs_tourney_team_adj_offense'})

In [54]:
tourney_games = tourney_games.merge(vs_tourney_team_adj_offense, on=['team', 'season'], how='left')

In [55]:
vs_tourney_team_adj_offense.rename(columns={'vs_tourney_team_adj_offense': 'opponent_vs_tourney_team_adj_offense', 'team':'opponent'}, inplace=True)

In [56]:
tourney_games = tourney_games.merge(vs_tourney_team_adj_offense, on=['opponent', 'season'], how='left')

In [57]:
vs_tourney_team_adj_offense_std = games_vs_tourney_teams[['team', 'season','adj_offense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_offense': 'vs_tourney_team_adj_offense_std'})
tourney_games = tourney_games.merge(vs_tourney_team_adj_offense_std, on=['team', 'season'], how='left')
vs_tourney_team_adj_offense_std.rename(columns={'vs_tourney_team_adj_offense_std': 'opponent_vs_tourney_team_adj_offense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_adj_offense_std, on=['opponent', 'season'], how='left')

In [58]:
vs_tourney_team_adj_defense = games_vs_tourney_teams[['team', 'season','adj_defense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_defense': 'vs_tourney_team_adj_defense'})

In [59]:
tourney_games = tourney_games.merge(vs_tourney_team_adj_defense, on=['team', 'season'], how='left')

In [60]:
vs_tourney_team_adj_defense.rename(columns={'vs_tourney_team_adj_defense': 'opponent_vs_tourney_team_adj_defense', 'team':'opponent'}, inplace=True)

In [61]:
tourney_games = tourney_games.merge(vs_tourney_team_adj_defense, on=['opponent', 'season'], how='left')

In [62]:
vs_tourney_team_adj_defense_std = games_vs_tourney_teams[['team', 'season','adj_defense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_defense': 'vs_tourney_team_adj_defense_std'})
tourney_games = tourney_games.merge(vs_tourney_team_adj_defense_std, on=['team', 'season'], how='left')
vs_tourney_team_adj_defense_std.rename(columns={'vs_tourney_team_adj_defense_std': 'opponent_vs_tourney_team_adj_defense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_adj_defense_std, on=['opponent', 'season'], how='left')

In [63]:
last_10_adj_defense = last_10_non_tourney[['team', 'season','adj_defense']].groupby(['team', 'season']).mean().reset_index().rename(columns={'adj_defense': 'last_10_adj_defense'})
tourney_games = tourney_games.merge(last_10_adj_defense, on=['team', 'season'], how='left')
last_10_adj_defense.rename(columns={'last_10_adj_defense': 'opponent_last_10_adj_defense', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_adj_defense, on=['opponent', 'season'], how='left')

In [64]:
last_10_adj_defense_std = last_10_non_tourney[['team', 'season','adj_defense']].groupby(['team', 'season']).std().reset_index().rename(columns={'adj_defense': 'last_10_adj_defense_std'})
tourney_games = tourney_games.merge(last_10_adj_defense_std, on=['team', 'season'], how='left')
last_10_adj_defense_std.rename(columns={'last_10_adj_defense_std': 'opponent_last_10_adj_defense_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_adj_defense_std, on=['opponent', 'season'], how='left')

In [65]:
season_offensive_efficiency = pre_tourney_games[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_efficiency': 'season_offensive_efficiency'})

In [66]:
tourney_games = tourney_games.merge(season_offensive_efficiency, on=['team', 'season'], how='left')

In [67]:
season_offensive_efficiency.rename(columns={'season_offensive_efficiency': 'opponent_season_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_offensive_efficiency, on=['opponent', 'season'], how='left')

In [68]:
season_offensive_efficiency_std = pre_tourney_games[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_efficiency': 'season_offensive_efficiency_std'})
tourney_games = tourney_games.merge(season_offensive_efficiency_std, on=['team', 'season'], how='left')
season_offensive_efficiency_std.rename(columns={'season_offensive_efficiency_std': 'opponent_season_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [69]:
vs_tourney_team_offensive_efficiency = games_vs_tourney_teams[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_efficiency': 'vs_tourney_team_offensive_efficiency'})

In [70]:
tourney_games = tourney_games.merge(vs_tourney_team_offensive_efficiency, on=['team', 'season'], how='left')

In [71]:
vs_tourney_team_offensive_efficiency.rename(columns={'vs_tourney_team_offensive_efficiency': 'opponent_vs_tourney_team_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_offensive_efficiency, on=['opponent', 'season'], how='left')

In [72]:
vs_tourney_team_offensive_efficiency_std = games_vs_tourney_teams[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_efficiency': 'vs_tourney_team_offensive_efficiency_std'})
tourney_games = tourney_games.merge(vs_tourney_team_offensive_efficiency_std, on=['team', 'season'], how='left')
vs_tourney_team_offensive_efficiency_std.rename(columns={'vs_tourney_team_offensive_efficiency_std': 'opponent_vs_tourney_team_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [73]:
last_10_offensive_efficiency = last_10_non_tourney[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_efficiency': 'last_10_offensive_efficiency'})
tourney_games = tourney_games.merge(last_10_offensive_efficiency, on=['team', 'season'], how='left')
last_10_offensive_efficiency.rename(columns={'last_10_offensive_efficiency': 'opponent_last_10_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_offensive_efficiency, on=['opponent', 'season'], how='left')

In [74]:
last_10_offensive_efficiency_std = last_10_non_tourney[['team', 'season','offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_efficiency': 'last_10_offensive_efficiency_std'})
tourney_games = tourney_games.merge(last_10_offensive_efficiency_std, on=['team', 'season'], how='left')
last_10_offensive_efficiency_std.rename(columns={'last_10_offensive_efficiency_std': 'opponent_last_10_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [75]:
season_effective_fg_percent = pre_tourney_games[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'effective_fg_percent': 'season_effective_fg_percent'})

In [76]:
tourney_games = tourney_games.merge(season_effective_fg_percent, on=['team', 'season'], how='left')

In [77]:
season_effective_fg_percent.rename(columns={'season_effective_fg_percent': 'opponent_season_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_effective_fg_percent, on=['opponent', 'season'], how='left')

In [78]:
season_effective_fg_percent_std = pre_tourney_games[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'effective_fg_percent': 'season_effective_fg_percent_std'})
tourney_games = tourney_games.merge(season_effective_fg_percent_std, on=['team', 'season'], how='left')
season_effective_fg_percent_std.rename(columns={'season_effective_fg_percent_std': 'opponent_season_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [79]:
vs_tourney_team_effective_fg_percent = games_vs_tourney_teams[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'effective_fg_percent': 'vs_tourney_team_effective_fg_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_effective_fg_percent, on=['team', 'season'], how='left')
vs_tourney_team_effective_fg_percent.rename(columns={'vs_tourney_team_effective_fg_percent': 'opponent_vs_tourney_team_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_effective_fg_percent, on=['opponent', 'season'], how='left')

In [80]:
vs_tourney_team_effective_fg_percent_std = games_vs_tourney_teams[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'effective_fg_percent': 'vs_tourney_team_effective_fg_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_effective_fg_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_effective_fg_percent_std.rename(columns={'vs_tourney_team_effective_fg_percent_std': 'opponent_vs_tourney_team_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [81]:
last_10_effective_fg_percent = last_10_non_tourney[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'effective_fg_percent': 'last_10_effective_fg_percent'})
tourney_games = tourney_games.merge(last_10_effective_fg_percent, on=['team', 'season'], how='left')
last_10_effective_fg_percent.rename(columns={'last_10_effective_fg_percent': 'opponent_last_10_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_effective_fg_percent, on=['opponent', 'season'], how='left')

In [82]:
last_10_effective_fg_percent_std = last_10_non_tourney[['team', 'season','effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'effective_fg_percent': 'last_10_effective_fg_percent_std'})
tourney_games = tourney_games.merge(last_10_effective_fg_percent_std, on=['team', 'season'], how='left')
last_10_effective_fg_percent_std.rename(columns={'last_10_effective_fg_percent_std': 'opponent_last_10_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [83]:
season_turnover_percent = pre_tourney_games[['team', 'season','turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'turnover_percent': 'season_turnover_percent'})
tourney_games = tourney_games.merge(season_turnover_percent, on=['team', 'season'], how='left')
season_turnover_percent.rename(columns={'season_turnover_percent': 'opponent_season_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_turnover_percent, on=['opponent', 'season'], how='left')

In [84]:
season_turnover_percent_std = pre_tourney_games[['team', 'season','turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'turnover_percent': 'season_turnover_percent_std'})
tourney_games = tourney_games.merge(season_turnover_percent_std, on=['team', 'season'], how='left')
season_turnover_percent_std.rename(columns={'season_turnover_percent_std': 'opponent_season_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_turnover_percent_std, on=['opponent', 'season'], how='left')

In [85]:
vs_tourney_team_turnover_percent = games_vs_tourney_teams[['team', 'season','turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'turnover_percent': 'vs_tourney_team_turnover_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_turnover_percent, on=['team', 'season'], how='left')
vs_tourney_team_turnover_percent.rename(columns={'vs_tourney_team_turnover_percent': 'opponent_vs_tourney_team_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_turnover_percent, on=['opponent', 'season'], how='left')

In [86]:
vs_tourney_team_turnover_percent_std = games_vs_tourney_teams[['team', 'season','turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'turnover_percent': 'vs_tourney_team_turnover_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_turnover_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_turnover_percent_std.rename(columns={'vs_tourney_team_turnover_percent_std': 'opponent_vs_tourney_team_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_turnover_percent_std, on=['opponent', 'season'], how='left')

In [87]:
last_10_turnover_percent = last_10_non_tourney[['team', 'season','turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'turnover_percent': 'last_10_turnover_percent'})
tourney_games = tourney_games.merge(last_10_turnover_percent, on=['team', 'season'], how='left')
last_10_turnover_percent.rename(columns={'last_10_turnover_percent': 'opponent_last_10_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_turnover_percent, on=['opponent', 'season'], how='left')

In [88]:
last_10_turnover_percent_std = last_10_non_tourney[['team', 'season','turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'turnover_percent': 'last_10_turnover_percent_std'})
tourney_games = tourney_games.merge(last_10_turnover_percent_std, on=['team', 'season'], how='left')
last_10_turnover_percent_std.rename(columns={'last_10_turnover_percent_std': 'opponent_last_10_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_turnover_percent_std, on=['opponent', 'season'], how='left')

In [89]:
season_offensive_rebound_percent = pre_tourney_games[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_rebound_percent': 'season_offensive_rebound_percent'})
tourney_games = tourney_games.merge(season_offensive_rebound_percent, on=['team', 'season'], how='left')
season_offensive_rebound_percent.rename(columns={'season_offensive_rebound_percent': 'opponent_season_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [90]:
season_offensive_rebound_percent_std = pre_tourney_games[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_rebound_percent': 'season_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(season_offensive_rebound_percent_std, on=['team', 'season'], how='left')
season_offensive_rebound_percent_std.rename(columns={'season_offensive_rebound_percent_std': 'opponent_season_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [91]:
vs_tourney_team_offensive_rebound_percent = games_vs_tourney_teams[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_rebound_percent': 'vs_tourney_team_offensive_rebound_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_offensive_rebound_percent, on=['team', 'season'], how='left')
vs_tourney_team_offensive_rebound_percent.rename(columns={'vs_tourney_team_offensive_rebound_percent': 'opponent_vs_tourney_team_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [92]:
vs_tourney_team_offensive_rebound_percent_std = games_vs_tourney_teams[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_rebound_percent': 'vs_tourney_team_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_offensive_rebound_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_offensive_rebound_percent_std.rename(columns={'vs_tourney_team_offensive_rebound_percent_std': 'opponent_vs_tourney_team_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [93]:
last_10_offensive_rebound_percent = last_10_non_tourney[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'offensive_rebound_percent': 'last_10_offensive_rebound_percent'})
tourney_games = tourney_games.merge(last_10_offensive_rebound_percent, on=['team', 'season'], how='left')
last_10_offensive_rebound_percent.rename(columns={'last_10_offensive_rebound_percent': 'opponent_last_10_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [94]:
last_10_offensive_rebound_percent_std = last_10_non_tourney[['team', 'season','offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'offensive_rebound_percent': 'last_10_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(last_10_offensive_rebound_percent_std, on=['team', 'season'], how='left')
last_10_offensive_rebound_percent_std.rename(columns={'last_10_offensive_rebound_percent_std': 'opponent_last_10_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [95]:
season_free_throw_rate = pre_tourney_games[['team', 'season','free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'free_throw_rate': 'season_free_throw_rate'})
tourney_games = tourney_games.merge(season_free_throw_rate, on=['team', 'season'], how='left')
season_free_throw_rate.rename(columns={'season_free_throw_rate': 'opponent_season_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_free_throw_rate, on=['opponent', 'season'], how='left')

In [96]:
season_free_throw_rate_std = pre_tourney_games[['team', 'season','free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'free_throw_rate': 'season_free_throw_rate_std'})
tourney_games = tourney_games.merge(season_free_throw_rate_std, on=['team', 'season'], how='left')
season_free_throw_rate_std.rename(columns={'season_free_throw_rate_std': 'opponent_season_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [97]:
vs_tourney_team_free_throw_rate = games_vs_tourney_teams[['team', 'season','free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'free_throw_rate': 'vs_tourney_team_free_throw_rate'})
tourney_games = tourney_games.merge(vs_tourney_team_free_throw_rate, on=['team', 'season'], how='left')
vs_tourney_team_free_throw_rate.rename(columns={'vs_tourney_team_free_throw_rate': 'opponent_vs_tourney_team_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_free_throw_rate, on=['opponent', 'season'], how='left')

In [98]:
vs_tourney_team_free_throw_rate_std = games_vs_tourney_teams[['team', 'season','free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'free_throw_rate': 'vs_tourney_team_free_throw_rate_std'})
tourney_games = tourney_games.merge(vs_tourney_team_free_throw_rate_std, on=['team', 'season'], how='left')
vs_tourney_team_free_throw_rate_std.rename(columns={'vs_tourney_team_free_throw_rate_std': 'opponent_vs_tourney_team_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [99]:
last_10_free_throw_rate = last_10_non_tourney[['team', 'season','free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'free_throw_rate': 'last_10_free_throw_rate'})
tourney_games = tourney_games.merge(last_10_free_throw_rate, on=['team', 'season'], how='left')
last_10_free_throw_rate.rename(columns={'last_10_free_throw_rate': 'opponent_last_10_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_free_throw_rate, on=['opponent', 'season'], how='left')

In [100]:
last_10_free_throw_rate_std = last_10_non_tourney[['team', 'season','free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'free_throw_rate': 'last_10_free_throw_rate_std'})
tourney_games = tourney_games.merge(last_10_free_throw_rate_std, on=['team', 'season'], how='left')
last_10_free_throw_rate_std.rename(columns={'last_10_free_throw_rate_std': 'opponent_last_10_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [101]:
season_allowed_offensive_efficiency = pre_tourney_games[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_efficiency': 'season_allowed_offensive_efficiency'})
tournament_games = tourney_games.merge(season_allowed_offensive_efficiency, on=['team', 'season'], how='left')
season_allowed_offensive_efficiency.rename(columns={'season_allowed_offensive_efficiency': 'opponent_season_allowed_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_offensive_efficiency, on=['opponent', 'season'], how='left')

In [102]:
season_allowed_offensive_efficiency_std = pre_tourney_games[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_efficiency': 'season_allowed_offensive_efficiency_std'})
tourney_games = tourney_games.merge(season_allowed_offensive_efficiency_std, on=['team', 'season'], how='left')
season_allowed_offensive_efficiency_std.rename(columns={'season_allowed_offensive_efficiency_std': 'opponent_season_allowed_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [103]:
vs_tourney_team_allowed_offensive_efficiency = games_vs_tourney_teams[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_efficiency': 'vs_tourney_team_allowed_offensive_efficiency'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_efficiency, on=['team', 'season'], how='left')
vs_tourney_team_allowed_offensive_efficiency.rename(columns={'vs_tourney_team_allowed_offensive_efficiency': 'opponent_vs_tourney_team_allowed_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_efficiency, on=['opponent', 'season'], how='left')

In [104]:
vs_tourney_team_allowed_offensive_efficiency_std = games_vs_tourney_teams[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_efficiency': 'vs_tourney_team_allowed_offensive_efficiency_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_efficiency_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_offensive_efficiency_std.rename(columns={'vs_tourney_team_allowed_offensive_efficiency_std': 'opponent_vs_tourney_team_allowed_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [105]:
last_10_allowed_offensive_efficiency = last_10_non_tourney[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_efficiency': 'last_10_allowed_offensive_efficiency'})
tourney_games = tourney_games.merge(last_10_allowed_offensive_efficiency, on=['team', 'season'], how='left')
last_10_allowed_offensive_efficiency.rename(columns={'last_10_allowed_offensive_efficiency': 'opponent_last_10_allowed_offensive_efficiency', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_offensive_efficiency, on=['opponent', 'season'], how='left')

In [106]:
last_10_allowed_offensive_efficiency_std = last_10_non_tourney[['team', 'season','opponent_offensive_efficiency']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_efficiency': 'last_10_allowed_offensive_efficiency_std'})
tourney_games = tourney_games.merge(last_10_allowed_offensive_efficiency_std, on=['team', 'season'], how='left')
last_10_allowed_offensive_efficiency_std.rename(columns={'last_10_allowed_offensive_efficiency_std': 'opponent_last_10_allowed_offensive_efficiency_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_offensive_efficiency_std, on=['opponent', 'season'], how='left')

In [107]:
season_allowed_effective_fg_percent = pre_tourney_games[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_effective_fg_percent': 'season_allowed_effective_fg_percent'})
tourney_games = tourney_games.merge(season_allowed_effective_fg_percent, on=['team', 'season'], how='left')
season_allowed_effective_fg_percent.rename(columns={'season_allowed_effective_fg_percent': 'opponent_season_allowed_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_effective_fg_percent, on=['opponent', 'season'], how='left')

In [108]:
season_allowed_effective_fg_percent_std = pre_tourney_games[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_effective_fg_percent': 'season_allowed_effective_fg_percent_std'})
tourney_games = tourney_games.merge(season_allowed_effective_fg_percent_std, on=['team', 'season'], how='left')
season_allowed_effective_fg_percent_std.rename(columns={'season_allowed_effective_fg_percent_std': 'opponent_season_allowed_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [109]:
vs_tourney_team_allowed_effective_fg_percent = games_vs_tourney_teams[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_effective_fg_percent': 'vs_tourney_team_allowed_effective_fg_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_effective_fg_percent, on=['team', 'season'], how='left')
vs_tourney_team_allowed_effective_fg_percent.rename(columns={'vs_tourney_team_allowed_effective_fg_percent': 'opponent_vs_tourney_team_allowed_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_effective_fg_percent, on=['opponent', 'season'], how='left')

In [110]:
vs_tourney_team_allowed_effective_fg_percent_std = games_vs_tourney_teams[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_effective_fg_percent': 'vs_tourney_team_allowed_effective_fg_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_effective_fg_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_effective_fg_percent_std.rename(columns={'vs_tourney_team_allowed_effective_fg_percent_std': 'opponent_vs_tourney_team_allowed_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [111]:
last_10_allowed_effective_fg_percent = last_10_non_tourney[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_effective_fg_percent': 'last_10_allowed_effective_fg_percent'})
tourney_games = tourney_games.merge(last_10_allowed_effective_fg_percent, on=['team', 'season'], how='left')
last_10_allowed_effective_fg_percent.rename(columns={'last_10_allowed_effective_fg_percent': 'opponent_last_10_allowed_effective_fg_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_effective_fg_percent, on=['opponent', 'season'], how='left')

In [112]:
last_10_allowed_effective_fg_percent_std = last_10_non_tourney[['team', 'season','opponent_effective_fg_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_effective_fg_percent': 'last_10_allowed_effective_fg_percent_std'})
tourney_games = tourney_games.merge(last_10_allowed_effective_fg_percent_std, on=['team', 'season'], how='left')
last_10_allowed_effective_fg_percent_std.rename(columns={'last_10_allowed_effective_fg_percent_std': 'opponent_last_10_allowed_effective_fg_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_effective_fg_percent_std, on=['opponent', 'season'], how='left')

In [113]:
season_allowed_turnover_percent = pre_tourney_games[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_turnover_percent': 'season_allowed_turnover_percent'})
tourney_games = tourney_games.merge(season_allowed_turnover_percent, on=['team', 'season'], how='left')
season_allowed_turnover_percent.rename(columns={'season_allowed_turnover_percent': 'opponent_season_allowed_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_turnover_percent, on=['opponent', 'season'], how='left')

In [114]:
season_allowed_turnover_percent_std = pre_tourney_games[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_turnover_percent': 'season_allowed_turnover_percent_std'})
tourney_games = tourney_games.merge(season_allowed_turnover_percent_std, on=['team', 'season'], how='left')
season_allowed_turnover_percent_std.rename(columns={'season_allowed_turnover_percent_std': 'opponent_season_allowed_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_turnover_percent_std, on=['opponent', 'season'], how='left')

In [115]:
vs_tourney_team_allowed_turnover_percent = games_vs_tourney_teams[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_turnover_percent': 'vs_tourney_team_allowed_turnover_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_turnover_percent, on=['team', 'season'], how='left')
vs_tourney_team_allowed_turnover_percent.rename(columns={'vs_tourney_team_allowed_turnover_percent': 'opponent_vs_tourney_team_allowed_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_turnover_percent, on=['opponent', 'season'], how='left')

In [116]:
vs_tourney_team_allowed_turnover_percent_std = games_vs_tourney_teams[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_turnover_percent': 'vs_tourney_team_allowed_turnover_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_turnover_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_turnover_percent_std.rename(columns={'vs_tourney_team_allowed_turnover_percent_std': 'opponent_vs_tourney_team_allowed_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_turnover_percent_std, on=['opponent', 'season'], how='left')

In [117]:
last_10_allowed_turnover_percent = last_10_non_tourney[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_turnover_percent': 'last_10_allowed_turnover_percent'})
tourney_games = tourney_games.merge(last_10_allowed_turnover_percent, on=['team', 'season'], how='left')
last_10_allowed_turnover_percent.rename(columns={'last_10_allowed_turnover_percent': 'opponent_last_10_allowed_turnover_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_turnover_percent, on=['opponent', 'season'], how='left')

In [118]:
last_10_allowed_turnover_percent_std = last_10_non_tourney[['team', 'season','opponent_turnover_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_turnover_percent': 'last_10_allowed_turnover_percent_std'})
tourney_games = tourney_games.merge(last_10_allowed_turnover_percent_std, on=['team', 'season'], how='left')
last_10_allowed_turnover_percent_std.rename(columns={'last_10_allowed_turnover_percent_std': 'opponent_last_10_allowed_turnover_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_turnover_percent_std, on=['opponent', 'season'], how='left')

In [119]:
season_allowed_offensive_rebound_percent = pre_tourney_games[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'season_allowed_offensive_rebound_percent'})
tourney_games = tourney_games.merge(season_allowed_offensive_rebound_percent, on=['team', 'season'], how='left')
season_allowed_offensive_rebound_percent.rename(columns={'season_allowed_offensive_rebound_percent': 'opponent_season_allowed_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [120]:
season_allowed_offensive_rebound_percent_std = pre_tourney_games[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'season_allowed_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(season_allowed_offensive_rebound_percent_std, on=['team', 'season'], how='left')
season_allowed_offensive_rebound_percent_std.rename(columns={'season_allowed_offensive_rebound_percent_std': 'opponent_season_allowed_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [121]:
vs_tourney_team_allowed_offensive_rebound_percent = games_vs_tourney_teams[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'vs_tourney_team_allowed_offensive_rebound_percent'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_rebound_percent, on=['team', 'season'], how='left')
vs_tourney_team_allowed_offensive_rebound_percent.rename(columns={'vs_tourney_team_allowed_offensive_rebound_percent': 'opponent_vs_tourney_team_allowed_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [122]:
vs_tourney_team_allowed_offensive_rebound_percent_std = games_vs_tourney_teams[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'vs_tourney_team_allowed_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_rebound_percent_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_offensive_rebound_percent_std.rename(columns={'vs_tourney_team_allowed_offensive_rebound_percent_std': 'opponent_vs_tourney_team_allowed_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [123]:
last_10_allowed_offensive_rebound_percent = last_10_non_tourney[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'last_10_allowed_offensive_rebound_percent'})
tourney_games = tourney_games.merge(last_10_allowed_offensive_rebound_percent, on=['team', 'season'], how='left')
last_10_allowed_offensive_rebound_percent.rename(columns={'last_10_allowed_offensive_rebound_percent': 'opponent_last_10_allowed_offensive_rebound_percent', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_offensive_rebound_percent, on=['opponent', 'season'], how='left')

In [124]:
last_10_allowed_offensive_rebound_percent_std = last_10_non_tourney[['team', 'season','opponent_offensive_rebound_percent']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_offensive_rebound_percent': 'last_10_allowed_offensive_rebound_percent_std'})
tourney_games = tourney_games.merge(last_10_allowed_offensive_rebound_percent_std, on=['team', 'season'], how='left')
last_10_allowed_offensive_rebound_percent_std.rename(columns={'last_10_allowed_offensive_rebound_percent_std': 'opponent_last_10_allowed_offensive_rebound_percent_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_offensive_rebound_percent_std, on=['opponent', 'season'], how='left')

In [125]:
season_allowed_free_throw_rate = pre_tourney_games[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_free_throw_rate': 'season_allowed_free_throw_rate'})
tourney_games = tourney_games.merge(season_allowed_free_throw_rate, on=['team', 'season'], how='left')
season_allowed_free_throw_rate.rename(columns={'season_allowed_free_throw_rate': 'opponent_season_allowed_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_free_throw_rate, on=['opponent', 'season'], how='left')

In [126]:
season_allowed_free_throw_rate_std = pre_tourney_games[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_free_throw_rate': 'season_allowed_free_throw_rate_std'})
tourney_games = tourney_games.merge(season_allowed_free_throw_rate_std, on=['team', 'season'], how='left')
season_allowed_free_throw_rate_std.rename(columns={'season_allowed_free_throw_rate_std': 'opponent_season_allowed_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(season_allowed_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [127]:
vs_tourney_team_allowed_free_throw_rate = games_vs_tourney_teams[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_free_throw_rate': 'vs_tourney_team_allowed_free_throw_rate'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_free_throw_rate, on=['team', 'season'], how='left')
vs_tourney_team_allowed_free_throw_rate.rename(columns={'vs_tourney_team_allowed_free_throw_rate': 'opponent_vs_tourney_team_allowed_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_free_throw_rate, on=['opponent', 'season'], how='left')

In [128]:
vs_tourney_team_allowed_free_throw_rate_std = games_vs_tourney_teams[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_free_throw_rate': 'vs_tourney_team_allowed_free_throw_rate_std'})
tourney_games = tourney_games.merge(vs_tourney_team_allowed_free_throw_rate_std, on=['team', 'season'], how='left')
vs_tourney_team_allowed_free_throw_rate_std.rename(columns={'vs_tourney_team_allowed_free_throw_rate_std': 'opponent_vs_tourney_team_allowed_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(vs_tourney_team_allowed_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [129]:
last_10_allowed_free_throw_rate = last_10_non_tourney[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).mean().reset_index().rename(columns={'opponent_free_throw_rate': 'last_10_allowed_free_throw_rate'})
tourney_games = tourney_games.merge(last_10_allowed_free_throw_rate, on=['team', 'season'], how='left')
last_10_allowed_free_throw_rate.rename(columns={'last_10_allowed_free_throw_rate': 'opponent_last_10_allowed_free_throw_rate', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_free_throw_rate, on=['opponent', 'season'], how='left')

In [130]:
last_10_allowed_free_throw_rate_std = last_10_non_tourney[['team', 'season','opponent_free_throw_rate']].groupby(['team', 'season']).std().reset_index().rename(columns={'opponent_free_throw_rate': 'last_10_allowed_free_throw_rate_std'})
tourney_games = tourney_games.merge(last_10_allowed_free_throw_rate_std, on=['team', 'season'], how='left')
last_10_allowed_free_throw_rate_std.rename(columns={'last_10_allowed_free_throw_rate_std': 'opponent_last_10_allowed_free_throw_rate_std', 'team':'opponent'}, inplace=True)
tourney_games = tourney_games.merge(last_10_allowed_free_throw_rate_std, on=['opponent', 'season'], how='left')

In [131]:
tourney_games

,date,season,team,opponent,Win_Loss,regular_season_win_pct,opponent_regular_season_win_pct,season_win_pct,opponent_season_win_pct,season_ppg,...,season_allowed_free_throw_rate_std,opponent_season_allowed_free_throw_rate_std,vs_tourney_team_allowed_free_throw_rate,opponent_vs_tourney_team_allowed_free_throw_rate,vs_tourney_team_allowed_free_throw_rate_std,opponent_vs_tourney_team_allowed_free_throw_rate_std,last_10_allowed_free_throw_rate,opponent_last_10_allowed_free_throw_rate,last_10_allowed_free_throw_rate_std,opponent_last_10_allowed_free_throw_rate_std
0,3/18/08,2008,Coppin St.,Mount St. Mary's,L,0.333333,0.517241,0.411765,0.562500,59.147059,...,14.861917,16.578073,42.050000,39.816667,24.764874,22.945711,44.11,34.35,11.896820,10.018899
1,3/18/08,2008,Mount St. Mary's,Coppin St.,W,0.517241,0.333333,0.562500,0.411765,69.062500,...,16.578073,14.861917,39.816667,42.050000,22.945711,24.764874,34.35,44.11,10.018899,11.896820
2,3/19/08,2008,Utah,UTEP,W,0.535714,0.586207,0.533333,0.593750,67.933333,...,15.955692,20.243507,38.650000,58.100000,14.450309,12.348117,48.50,44.81,20.289735,14.291602
3,3/19/08,2008,UTEP,Utah,L,0.586207,0.535714,0.593750,0.533333,76.812500,...,20.243507,15.955692,58.100000,38.650000,12.348117,14.450309,44.81,48.50,14.291602,20.289735
4,3/19/08,2008,UAB,VCU,W,0.709677,0.793103,0.687500,0.774194,74.125000,...,15.745990,14.562407,39.216667,49.075000,15.614790,10.535456,43.99,30.84,15.237997,8.393675
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2381,3/19/15,2015,Wofford,Arkansas,L,0.793103,0.774194,0.812500,0.764706,65.875000,...,18.369768,15.406261,40.980000,44.550000,9.949724,16.126889,29.09,42.86,11.664614,12.013529
2382,3/20/15,2015,Northern Iowa,Wyoming,W,0.900000,0.689655,0.909091,0.718750,65.363636,...,12.266034,11.951216,32.672727,36.316667,10.988639,14.345092,21.99,24.61,11.977335,6.923783
2383,3/20/15,2015,Wyoming,Northern Iowa,L,0.689655,0.900000,0.718750,0.909091,61.437500,...,11.951216,12.266034,36.316667,32.672727,14.345092,10.988639,24.61,21.99,6.923783,11.977335
2384,3/26/15,2015,Arizona,Xavier,W,0.903226,0.612903,0.911765,0.617647,76.441176,...,14.133054,16.421777,41.068750,34.765000,14.598206,17.383182,35.45,35.17,15.127330,15.894237


In [132]:
conn.rollback()

In [133]:
tourney_games.to_sql('tourney_games_through_2024_final', conn, if_exists='replace', index=False, schema='sports_data')

166

In [134]:
conn.commit()

In [135]:
tournament_games.shape

(2386, 118)